In [16]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import PyPDF2
import re
import io
import time

# https://maximumfun.org/podcasts/adventure-zone/?_post-type=transcript&_paged=1&_sort=date_asc

In [22]:
def save_text(url, dir='data/adventure_zone/files/'):
    filename = url.split('/')[-1][:-4]
    try:
        remote_file = requests.get(url).content
        # with open('test.pdf', 'wb') as file:
        #     file.write(remote_file)
        memory_file = io.BytesIO(remote_file)
        reader = PyPDF2.PdfFileReader(memory_file)
        with open(dir + filename +'.txt', 'w') as file:
            for page in reader.pages:
                file.writelines(page.extractText())
        return filename
    except:
        return f'Skipped {filename}'

def get_pdf_link(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    for link in soup.findAll('a'):
        if link and bool(re.search('Download', link.text)):
    
            return link['href']

def get_episodes(url):
    links = []
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    for header in soup.findAll('h4'):
        link = header.find('a')
        if link and bool(re.search('TRANSCRIPT', link.text)):
    
            links.append(link['href'])
    return links

# Main Loop

In [24]:
def main():
    for i in range(1,16):
        url = f'https://maximumfun.org/podcasts/adventure-zone/?_post-type=transcript&_paged={i}&_sort=date_asc'
        # print(url)
        episodes = get_episodes(url)
        # print(episodes)

        for url in episodes:
            url = get_pdf_link(url)
            filename = save_text(url)
            print(f'{i}/{15}:\t{filename}',end='\r',flush=True)
            time.sleep(0.1)

In [25]:
main()